# CRAWLER AND SCRAPER OF ACM DIGITAL LIBRARY

## NEEDED LIBRARIES

In [ ]:
import pandas as pd
import cfscrape
from lxml import etree

## SET OF KEYWORDS

In [ ]:
keywords = ['Cloud Security']

## INSTANTIATING THE CRAWLER AND DEFINING AN REALISTIC HEADER 

In [ ]:
scraper = cfscrape.create_scraper()

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'}

## READING OLD DATASET FOR APPENDING or CREATE A NEW (EMPTY) DATAFRAME

In [ ]:
# df= pd.read_csv('20191001_acm_dl_ddos_papers.csv').drop(['Unnamed: 0'],axis=1)
df = pd.DataFrame(columns=['links'])

## LOOP FOR CRAWLING AND SCRAPPING (IN 2 LEVELS) ALL ENTRIES RELATED TO THE SET OF KEYWORDS 

In [ ]:
for keyword in keywords:
    page = 1 
    while True:
        print(page,keyword)
        
        keyword = keyword.replace(' ','+')
        url = "https://dl.acm.org/results.cfm?query=%22"+keyword+"%22&start="+str((page-1)*20)+"&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"
        print(url)
        
        scraped_html=scraper.get(url,headers=header).content
        html = etree.HTML(scraped_html)
        
        max_entries = html.xpath("//div[@class='pagerange']/text()")[0].split()[-1]
        if (page-1)*20 > int(max_entries):
            break
        
        df_temp = pd.DataFrame()
        
        #Crawler first level
        title = []
        links = []
        publication_date = []
        venue=[]

        for entry in html.xpath("//div[@class='title']/a"):
            try:
                title.append(entry.xpath("text()")[0])
            except:
                title.append("")

            try: 
                links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
            except:
                links.append("")

        for entry in html.xpath("//div[@class='source']"): 
            try:
                publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
            except:
                publication_date.append("")

            try:
                venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
            except:
                venue.append("")

        df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})
    
        #Crawler second level
        for link in df_temp['links']:
            print(link)
            # CHECK IF THE LINK ALREADY EXIST
            if len(df[df['links']==link]) >0:
                df_temp.drop(df_temp.loc[df_temp['links']==link].index, inplace=True)
                print("duplicated")
                continue
                
            
            scraped_html=scraper.get(link,headers=header).content
            html = etree.HTML(scraped_html)

            index_current = df_temp.index[df_temp['links'] == link][0]

            try:
                df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
            except:
                df_temp.at[index_current, 'link4download'] = ""

            try:
                df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
            except:
                df_temp.at[index_current, 'num_citations'] =""

            try:
                df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
            except:
                df_temp.at[index_current, 'num_downloads_cu'] =""

            try:
                df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
            except:
                try: 
                    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
                except:
                    try:
                        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
                    except:
                        df_temp.at[index_current, 'abstract'] = ""
        
        df = df.append(df_temp,ignore_index=True)
        display(df.tail(1))
        
        page = page + 1

In [ ]:
df.to_csv('20191111_acm_dl_cloud_security.csv')

<h1 align='center'>============================================================</h1>

## DEBUGGING ERROS 1ST LEVEL CRAWLER

In [ ]:
url="https://dl.acm.org/results.cfm?query=%22denial+of+service%22&start=300&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"

In [ ]:
scraped_html=scraper.get(url,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
title = []
links = []
publication_date = []
venue=[]

for entry in html.xpath("//div[@class='title']/a"):
    try:
        title.append(entry.xpath("text()")[0])
    except:
        title.append("")
    
    try: 
        links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
    except:
        links.append("")

for entry in html.xpath("//div[@class='source']"): 
    try:
        publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
    except:
        publication_date.append("")
    
    try:
        venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
    except:
        venue.append("")

df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})

## DEBUGING ERROS 2nd LEVEL CRAWLER

In [ ]:
link = "https://dl.acm.org/citation.cfm?id=372148&preflayout=flat"

In [ ]:
scraped_html=scraper.get(link,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
index_current = 0

try:
    df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
except:
    df_temp.at[index_current, 'link4download'] = ""

try:
    df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
except:
    df_temp.at[index_current, 'num_citations'] =""

try:
    df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
except:
    df_temp.at[index_current, 'num_downloads_cu'] =""

try:
    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
except:
    try: 
        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
    except:
        try:
            df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
        except:
            df_temp.at[index_current, 'abstract'] = ""

df_temp

<h1 align='center'>###############################################################<br>OFFLINE PROCESSING THE DATA</h1>

In [122]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)

## READING THE DATAFRAME 

In [85]:
df= pd.read_csv('20191111_acm_dl_cloud_security.csv',index_col=0)

## QUICK PREVIEW OF THE DATA

In [86]:
df.head()

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
0,Most of the problems facing the designers of a...,https://dl.acm.org/ft_gateway.cfm?id=2046668&f...,https://dl.acm.org/citation.cfm?id=2046668&pre...,0,"2,103",October 2011,What's different about security in a public cl...,CCSW '11: Proceedings of the 3rd ACM workshop ...
1,Cloud computing has become increasingly popula...,https://dl.acm.org/ft_gateway.cfm?id=2046662&f...,https://dl.acm.org/citation.cfm?id=2046662&pre...,0,845,October 2011,Clouds and their discontents,CCSW '11: Proceedings of the 3rd ACM workshop ...
2,Can the cloud truly be secured? Can enterprise...,https://dl.acm.org/ft_gateway.cfm?id=2046674&f...,https://dl.acm.org/citation.cfm?id=2046674&pre...,0,"1,975",October 2011,Cloud security: myth or reality?,CCSW '11: Proceedings of the 3rd ACM workshop ...
3,Cloud computing is a long fantasized visualiza...,https://dl.acm.org/ft_gateway.cfm?id=2331767&f...,https://dl.acm.org/citation.cfm?id=2331767&pre...,6,"2,259",March 2012,Different facets of security in the cloud,CNS '12: Proceedings of the 15th Communication...
4,It is recognized that the most important obsta...,https://dl.acm.org/ft_gateway.cfm?id=2536164&f...,https://dl.acm.org/citation.cfm?id=2536164&pre...,1,341,October 2013,A general approach for a trusted deployment of...,MEDES '13: Proceedings of the Fifth Internatio...


## Showing most downloaded title of papers

In [ ]:
df['num_downloads_cu'] = pd.to_numeric(df['num_downloads_cu'].str.replace(',',''),errors='coerce')

In [121]:
df.sort_values(by='num_downloads_cu',ascending=False)['title'].head(20)

57     Cloud security: a gathering storm                                                                                                       
115    ACSAC '13: Proceedings of the 29th Annual Computer Security Applications Conference                                                     
88     Augmented-reality glasses bring cloud security into sharp focus                                                                         
141    CCSW '12: Proceedings of the 2012 ACM Workshop on Cloud computing security workshop                                                     
160    CloudDB '10: Proceedings of the second international workshop on Cloud data management                                                  
168    CCSW '14: Proceedings of the 6th edition of the ACM Workshop on Cloud Computing Security                                                
100    Cloud security is not (just) virtualization security: a short paper                                                              

## Count frequency of words in the title

In [95]:
title_word_freq = df['title'].str.lower().str.split(expand=True).stack().value_counts()

In [123]:
title_word_freq.head()

cloud       101
security    77 
the         59 
of          55 
for         53 
dtype: int64

## Removing stop-words

In [124]:
import nltk

In [125]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/nw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [126]:
title_word_freq_nostopwords = title_word_freq.drop(stopwords,errors='ignore')

In [127]:
title_word_freq_nostopwords.head(50)

cloud            101
security         77 
data             32 
computing        24 
secure           15 
proceedings      14 
workshop         13 
framework        13 
clouds           11 
privacy          10 
based            9  
system           9  
services         9  
encryption       8  
scheme           7  
international    7  
using            7  
cloud:           6  
control          6  
information      6  
private          6  
storage          6  
management       6  
securing         6  
acm              6  
applications     5  
analysis         5  
sharing          5  
systems          5  
service          5  
malware          5  
detection        5  
environments     5  
attacks          5  
auditing         4  
scc              4  
environment      4  
ccsw             4  
session          4  
challenges       4  
conference       4  
key              4  
authorization    4  
mobile           4  
protection       4  
dynamic          4  
encrypted        4  
security:    